In [1]:
import pandas as pd
tweets_preprocessed = pd.read_csv('samsung_balanced_11098_preprocessed.csv')
tweets_preprocessed.head(3)

,id,text,polarity,polarity_confidence,subjectivity,subjectivity_confidence,polarityNum
0,975487595282030592,s exact opposite current phone even let manual...,negative,0.908798,subjective,1.000000,-1.0
1,962319102806118400,samsung gifted free phone winter olympics athl...,negative,0.461419,subjective,1.000000,-1.0
2,926460821214961665,seriously aku nak buang samsung pay nie dari p...,negative,0.929383,objective,0.999998,-1.0


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBRegressor
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
train, test = train_test_split(tweets_preprocessed, test_size=0.2, random_state=1)
X_train = train['text'].values
X_test = test['text'].values
y_train = train['polarityNum']
y_test = test['polarityNum']

In [4]:
v = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=True, sublinear_tf=True, stop_words='english', analyzer='word', ngram_range = (2,2))
train_features_model = v.fit(X_train)
test_features = v.transform(X_test)
train_features = v.fit_transform(X_train)

In [ ]:
#hyper parameter tuning not perfect yet
#Classifiers = [
clf = KNeighborsClassifier()
tuned_parameters = [{'n_neighbors': [3,5,7],'weights': ['distance','uniform']}]

#clf = SVC()
#tuned_parameters=[{'kernel': ['rbf','linear'],'C': [0.01,0.1,1, 10]}]
    # clf = DecisionTreeClassifier()
    # tuned_parameters=[]
    # clf= RandomForestClassifier(n_estimators=200)
    # tuned_parameters=[]
    # clf= AdaBoostClassifier()
    # tuned_parameters=[]
    # clf=BernoulliNB()
    # tuned_parameters=[] 

In [ ]:
# E.g "LabelKFold", "LeaveOneOut", "LeaveOneLabelOut", etc.
inner_cv = KFold(n_splits=5)
#inner_cv=StratifiedKFold(n_splits=10)

classifier = GridSearchCV(clf, tuned_parameters, cv=inner_cv, scoring='f1_macro')
model = classifier.fit(train_features, y_train)
scores = classifier.best_score_
scores_std = classifier.cv_results_['std_test_score']
print('F1-macro of '+classifier.__class__.__name__+' is '+str(scores))
print('F1-macro std of '+classifier.__class__.__name__+' is '+ str(scores_std))

In [ ]:
Classifiers = [
    LogisticRegression(C=0.1,solver='liblinear',max_iter=200),
    KNeighborsClassifier(9),
    SVC( kernel="linear", class_weight="balanced", C=0.1, probability=True),
    #NuSVC(kernel="rbf", probability=True),
    XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27),
    DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5),
    RandomForestClassifier(n_estimators=500),
    AdaBoostClassifier(n_estimators=500),
    MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None),
    BernoulliNB()]